# Feature Extraction

In [87]:
import os
import mne 
import pywt
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.signal import welch
from scipy.integrate import simpson
from scipy.stats import skew

In [88]:
load_dir = r"C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\Epochs"
loaded_epoch_list = []

for filename in os.listdir(load_dir):
    if filename.endswith("_epo.fif"):
        load_path = os.path.join(load_dir, filename)
        epochs = mne.read_epochs(load_path, preload = True)
        loaded_epoch_list.append(epochs)

print("Epoch list loaded")

Reading C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\Epochs\ch_10_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =  -10000.00 ...   45000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\Epochs\ch_11_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =  -10000.00 ...   32000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\Epochs\ch_12_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =  -10000.00 ...   42000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Read

Isotrak not found
    Found the data of interest:
        t =  -10000.00 ...   54000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\Epochs\ch_14_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =  -10000.00 ...   24000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\Epochs\ch_15_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =  -10000.00 ...  135000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\Epochs\ch_16_epo.fif ...
Isot

In [89]:
def time_domain_features(epochs):
 

  features = [] # feature list
  epoch_data = epochs.get_data()
  n_epochs, n_channels, n_times = epoch_data.shape
  f_dict = {}
  for ch in range(n_channels): # Iterating through each channel
    data = epoch_data[0,ch,:]
    f_dict[f'ch{ch}'] = np.mean(data)                           # Index 0 = Mean
  features.append(f_dict)

  f_dict = {}
  for ch in range(n_channels):
    data = epoch_data[0,ch,:]                                   # Index 2 = STD
    f_dict[f'ch{ch}']  = np.std(data)
  features.append(f_dict)

  f_dict = {}
  for ch in range(n_channels):
    data = epoch_data[0,ch,:]
    f_dict[f'ch{ch}']  = stats.skew(data)                       # Index 3 = skew
  features.append(f_dict)

  f_dict = {}
  for ch in range(n_channels):
    data = epoch_data[0,ch,:]
    f_dict[f'ch{ch}']  = stats.kurtosis(data)                # Index 4 = Kurtosis
  features.append(f_dict) # Each element in the list is a feature dict of each channel

  return features

In [90]:

def BandPower(data, sfreq, band, window_sec = None, relative = False):
  if band == 'delta':
    low, high = 0.5, 4
  elif band == 'theta':
    low, high = 4, 8
  elif band == 'alpha':
    low, high = 8, 12
  elif band == 'beta':
    low, high = 12, 30
  elif band == 'gamma':
    low, high = 30, 100
  else:
    raise ValueError('Invalid band')

  nperseg = (2/low) * sfreq

  freq, psd = welch(data, sfreq, nperseg = nperseg)

  freq_res = freq[1] - freq[0]

  idx_band = np.logical_and(freq >= low, freq <= high)

  bp = simpson(psd[idx_band], dx = freq_res)

  if relative:
    bp = bp/simpson(psd, dx = freq_res)
  return bp


In [93]:
# change the frequency based separation: 

def time_features_aggregation(epoch):

    time_features = time_domain_features(epoch)

    stats = ['Mean','Std','Skew','Kurtosis']
   
   
    pd_list = []

    for j in range(len(stats)):
        c = pd.DataFrame.from_dict(time_features, columns = [stats[j]], orient= 'index')
        pd_list.append(c)
    print(pd_list)
    result_df = pd.concat(pd_list, axis = 1)
    return result_df

def band_feature_aggregation(epoch):
    band_list = ['alpha', 'beta', 'gamma', 'theta', 'delta']
    Band_feature = []
    epoch_data = epoch.get_data()
#channel_data =epoch_data[0,0,:]
    for j in range(len(band_list)):
        Band_dict = {}
        for i in range(23):
            d = BandPower(data = epoch_data[0,i,:],sfreq = 210 , band = band_list[j])
            Band_dict[f'ch{i}'] = d
        Band_feature.append(Band_dict)
    
# Creating Dataframe
    band_columns = []
    for i in range(len(band_list)):
        c = pd.DataFrame.from_dict(Band_feature[i], columns = [band_list[i]], 
                                   orient = 'index')
        band_columns.append(c)
    Band_feature_df = pd.concat(band_columns, axis = 1)
    return Band_feature_df

def Merge_csv(time_feature_df, Band_feature_df, index ):
    df_list = [time_feature_df, Band_feature_df]
    feature_vector_df = pd.concat(df_list, axis = 1)
    feature_vector_df.to_csv(rf"C:\Users\ADMIN\Documents\Summerof2024\Epilepsy_Detection\\Feature Vector\Ch{index}_feature_df.csv")
    return feature_vector_df

for i, item in enumerate(loaded_epoch_list):
   time_df = time_features_aggregation(item)
   band_df = band_feature_aggregation(item)
   Merge_csv(time_feature_df = time_df, Band_feature_df = band_df, index = i)
   

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1387 samples (6.605 s)

Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-pass filter from 12 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 231 samples (1.100 s)

Setting up band-pass filter from 30 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 102.50

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch_data = epochs.get_data()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch_data = epochs.get_data()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch_data = epochs.get_data()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of c

[              Mean
ch0   4.424395e-09
ch1  -5.358697e-11
ch2   2.127611e-09
ch3  -6.105303e-09
ch4   2.073286e-09
ch5   4.980435e-11
ch6  -8.631143e-10
ch7  -9.642172e-10
ch8  -2.963526e-09
ch9  -1.750362e-09
ch10  7.386753e-11
ch11  1.848814e-09
ch12 -2.273757e-09
ch13  1.708893e-10
ch14  3.157977e-09
ch15 -4.103839e-09
ch16 -5.165637e-11
ch17 -1.748643e-10
ch18 -2.127611e-09
ch19 -9.285262e-10
ch20 -5.970962e-09
ch21  1.758572e-09
ch22  3.157977e-09,            Std
ch0   0.000016
ch1   0.000058
ch2   0.000044
ch3   0.000027
ch4   0.000018
ch5   0.000024
ch6   0.000025
ch7   0.000026
ch8   0.000017
ch9   0.000015
ch10  0.000017
ch11  0.000016
ch12  0.000012
ch13  0.000018
ch14  0.000018
ch15  0.000017
ch16  0.000022
ch17  0.000023
ch18  0.000044
ch19  0.000038
ch20  0.000027
ch21  0.000012
ch22  0.000018,           Skew
ch0  -0.001515
ch1  -0.003013
ch2   0.002166
ch3   0.004749
ch4  -0.002104
ch5   0.000352
ch6   0.001146
ch7  -0.001088
ch8   0.001635
ch9   0.002971
ch10 -0.002537
c

...

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1387 samples (6.605 s)

Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-pass filter from 30 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 102.50 Hz)
- Filter length: 139 samples (0.662 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch_data = epochs.get_data()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch_data = epochs.get_data()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch_data = epochs.get_data()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21112\3693584950.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch_data = epochs.ge

[              Mean
ch0  -1.631096e-09
ch1   1.794357e-09
ch2   1.077860e-09
ch3   1.068359e-09
ch4   2.587945e-09
ch5   3.855144e-09
ch6  -1.180066e-09
ch7  -3.031683e-09
ch8   3.563586e-09
ch9   5.251898e-09
ch10  1.494690e-10
ch11 -6.077352e-09
ch12  3.343971e-09
ch13  4.673706e-09
ch14 -3.394605e-09
ch15 -1.777290e-09
ch16  3.749328e-09
ch17 -5.515126e-10
ch18 -1.077860e-09
ch19 -1.412261e-09
ch20 -8.280584e-10
ch21  7.867932e-09
ch22 -3.394605e-09
ch23 -3.232719e-09,            Std
ch0   0.000083
ch1   0.000109
ch2   0.000137
ch3   0.000086
ch4   0.000098
ch5   0.000093
ch6   0.000137
ch7   0.000095
ch8   0.000091
ch9   0.000102
ch10  0.000075
ch11  0.000112
ch12  0.000094
ch13  0.000096
ch14  0.000113
ch15  0.000093
ch16  0.000090
ch17  0.000075
ch18  0.000137
ch19  0.000082
ch20  0.000096
ch21  0.000062
ch22  0.000113
ch23  0.000031,           Skew
ch0   0.001071
ch1  -0.001318
ch2  -0.000494
ch3   0.002291
ch4  -0.004943
ch5  -0.001726
ch6   0.001938
ch7   0.000409
ch8   0.0060

In [100]:
for item in loaded_epoch_list:
    print(len(item.info['ch_names']))

23
28
28
28
28
38
28
28
28
28
23
28
28
28
23
23
23
23
23
23
23
23
23
24
